In [24]:
from functions import *
from music21 import converter, environment

import pickle

In [2]:
env = environment.Environment()
env['musicxmlPath'] = 'C:\\Program Files\\MuseScore 4\\bin\\MuseScore4.exe'  # Path to MuseScore executable
env['musescoreDirectPNGPath'] = 'C:\\Program Files\\MuseScore 4\\bin\\MuseScore4.exe'  # Path to MuseScore executable

In [ ]:

midi_file = '..\\(Extrapolated) Quartet for 2 Violins, Viola and Violoncello D major 3 Nocturne.xml'
midi_parsed = converter.parse(midi_file)
# midi_parsed.show()

In [ ]:
nmat, narr, sarr = parse_score_elements(midi_parsed)
ir_symbols = assign_ir_symbols(narr)
ir_nmat = ir_symbols_to_matrix(ir_symbols, nmat)
ir_nmat = assign_ir_pattern_indices(ir_nmat)
segments = segmentgestalt(ir_nmat)
processed_segments = preprocess_segments(segments)
visualize_notes_with_symbols(ir_symbols)
# midi_parsed.show()

In [8]:
def segments_to_distance_matrix(segments: list[pd.DataFrame], cores=None, checkpoint=1000):
    """
    Converts segments to a distance matrix using multiprocessing.

    Parameters:
    segments (list[pd.DataFrame]): A list of segmented DataFrames.
    cores (int): The number of CPU cores to use for multiprocessing (default is None).
    checkpoint (int): The number of individual segment to segment distance calculations
                        to perform before saving to a pickle

    Returns:
    np.ndarray: A distance matrix representing distances between segments.
    """
    if cores is not None and cores > cpu_count():
        raise ValueError(f"You don't have enough cores! Please specify a value within your system's number of "
                         f"cores. Core Count: {cpu_count()}")


    # check if pickle exists
    if os.path.exists("segment_pair_args.pkl") and os.path.exists("unfinished_matrix.pkl"):

        print("unfinished matrix and segment pair pickles exists, skipping initialization, loading existing matrix")
        
        # loading unfinished matrix pickle
        with open("unfinished_matrix.pkl", "rb") as f:
            distance_matrix = pickle.load(f)

        # loading tracking variables
        with open("segment_pair_args.pkl", "rb") as f:
            segment_pair_args = pickle.load(f)
        
        segment_pairs_left = len(segment_pair_args)

    else:
        seg_np = [segment.to_numpy() for segment in segments]
        num_segments = len(seg_np)
        distance_matrix = np.zeros((num_segments, num_segments))

        segment_pair_args = []
        for i in range(num_segments):
            for j in range(i + 1, num_segments):
                segment_pair_args.append((i, j, segments[i], segments[j]))

        segment_pairs_left = len(segment_pair_args)

        # pickle the args list for checkpointing
        with open("segment_pair_args.pkl", "wb") as f:
            pickle.dump(segment_pair_args, f)


    while segment_pairs_left > 0:

        # load first n amount of remaining pairs to compute
        with open("segment_pair_args.pkl", "rb") as f:
            segment_pair_args = pickle.load(f)
        
        with Manager() as manager:
            message_list = manager.list()

            def log_message(message):
                message_list.append(message)

            with Pool(cores) as pool:
                results = pool.map(worker.calculate_distance, segment_pair_args[:1000]) ### hardcoded checkpoint: 1000; i havent fixed this one simple slice index error

            for i, j, distance, message in results:
                distance_matrix[i, j] = distance
                distance_matrix[j, i] = distance  # Reflect along the diagonal
                log_message(message)

            for message in message_list:
                print(message)

        segment_pairs_left -= checkpoint
        print(f"{segment_pairs_left} unprocessed segment pairs left...")

        # pickle the unfinished matrix
        with open("unfinished_matrix.pkl", "wb") as f:
            pickle.dump(distance_matrix, f)

        # pickle remaning segment pairs to calculate for the next iteration
        with open("segment_pair_args.pkl", "wb") as f:
            pickle.dump(segment_pair_args[1000:], f) ### hardcoded checkpoint: 1000; i havent fixed this one simple slice index error
    
    return distance_matrix

# add an interrupt detection to avoid redundancy and possible errors

In [25]:
distance_matrix = segments_to_distance_matrix(processed_segments)

In [53]:
# forceload distance matrix
with open("unfinished_matrix.pkl", "rb") as f:
    distance_matrix = pickle.load(f)

# print(type(distance_matrix))
# print(distance_matrix[10])

In [54]:
# distance matrix to knn graph function
def distance_matrix_to_knn_graph(k: int, distance_matrix: np.array, graph_title: str,
                                 seed: int, iterations: int):
  knn_graph = kneighbors_graph(distance_matrix, n_neighbors=k, mode='connectivity')

  G = nx.from_scipy_sparse_array(knn_graph)

  # Detect if the graph is disjoint
  if not nx.is_connected(G):
      print("The KNN graph is disjoint. Ensuring connectivity...")

      # Calculate the connected components
      components = list(nx.connected_components(G))

      # Connect the components
      for i in range(len(components) - 1):
          min_dist = np.inf
          closest_pair = None
          for node1 in components[i]:
              for node2 in components[i + 1]:
                  dist = distance_matrix[node1, node2]
                  if dist < min_dist:
                      min_dist = dist
                      closest_pair = (node1, node2)

          # Add an edge between the closest pair of nodes from different components
          G.add_edge(closest_pair[0], closest_pair[1])

  # Plot the final connected graph
  pos = nx.spring_layout(G, seed=seed, iterations=iterations)
  nx.draw(G, node_size=50, pos=pos)
  plt.title(graph_title + f" (K={k})")
  plt.show()

In [55]:
# distance matrix to knn graph function
def distance_matrices_to_knn_graph(k: int, distance_matrix: np.array, graph_title: str,
                                 seed: int, iterations: int):
  knn_graph = kneighbors_graph(distance_matrix, n_neighbors=k, mode='connectivity')

  G = nx.from_scipy_sparse_array(knn_graph)

  # Detect if the graph is disjoint
  if not nx.is_connected(G):
      print("The KNN graph is disjoint. Ensuring connectivity...")

      # Calculate the connected components
      components = list(nx.connected_components(G))

      # Connect the components
      for i in range(len(components) - 1):
          min_dist = np.inf
          closest_pair = None
          for node1 in components[i]:
              for node2 in components[i + 1]:
                  dist = distance_matrix[node1, node2]
                  if dist < min_dist:
                      min_dist = dist
                      closest_pair = (node1, node2)

          # Add an edge between the closest pair of nodes from different components
          G.add_edge(closest_pair[0], closest_pair[1])

  # Plot the final connected graph
  pos = nx.spring_layout(G, seed=seed, iterations=iterations)
  nx.draw(G, node_size=50, pos=pos)
  plt.title(graph_title + f" (K={k})")
  plt.show()

In [ ]:
# show graph
distance_matrix_to_knn_graph(3, distance_matrix, "Bach Prelude in C", 9, 70)